In [2]:
import rul_datasets
import torch
import warnings
from catalyst import dl, utils
from torch import nn, optim
torch.manual_seed(42)

In [3]:
import torch
import torch.nn as nn


class Shcherbakov(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        # CNN feature extraction part
        self.feature_extractor = nn.Sequential()
        self.feature_extractor.add_module(
            'Conv1d_1', nn.Conv1d(in_channels, out_channels, kernel_size)
        )
        self.feature_extractor.add_module('ReLU_1', nn.ReLU())
        self.feature_extractor.add_module(
            'Conv1d_2', nn.Conv1d(out_channels, out_channels * 2, kernel_size - 2)
        )
        self.feature_extractor.add_module('ReLU_2', nn.ReLU())
        self.feature_extractor.add_module('Flatten', nn.Flatten(start_dim=1))
        self.feature_extractor.add_module(
            'MaxPool1d', nn.MaxPool1d(kernel_size=3)
        )

        self.lstm_1 = nn.LSTM(input_size=48, hidden_size=96, num_layers=50)
        self.lstm_2 = nn.LSTM(input_size=96, hidden_size=96, num_layers=50)
        self.dropout = nn.Dropout(p=0.2)
        self.num_layers = 50
        self.tanh = nn.Tanh()

        # Regressor part
        self.regressor = nn.Sequential()
        self.regressor.add_module('Linear_1', nn.Linear(48, 1))
        self.regressor.add_module('ReLU_2', nn.ReLU())

        # Loss function
        self.loss = nn.MSELoss()

    def forward(self, time_series):
        # CNN feature extraction
        features = self.feature_extractor(time_series)

        # LSTM layers
        h_0 = torch.zeros(self.num_layers, 96)
        c_0 = torch.zeros(self.num_layers, 96)
        output, (h_n, c_n) = self.lstm_1(features, (h_0, c_0))
        output = self.tanh(self.dropout(output))
        output, (h_n, c_n) = self.lstm_2(output, (h_n, c_n))
        output = self.tanh(self.dropout(output))
        preds = self.regressor(features)
        return preds

In [4]:
# read data
cmapss_fd1 = rul_datasets.CmapssReader(fd=1)
dm = rul_datasets.RulDataModule(cmapss_fd1, batch_size=32)
dm.prepare_data()
dm.setup()

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, predictions, targets):
        return torch.sqrt(self.mse(predictions, targets))

In [11]:
# create model
model = Shcherbakov(in_channels=14, out_channels=3, kernel_size=5)
criterion = nn.MSELoss()
RMSE_loss = torch.sqrt(criterion)
optimizer = optim.Adam(model.parameters(), lr=0.02)

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not MSELoss

In [6]:
loaders = {
    "train": dm.train_dataloader(),
    "val": dm.val_dataloader(),
    "test": dm.test_dataloader(),
}

In [7]:
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

In [8]:
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk=(1, 3, 5)),
        dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets"),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
)

1/1 * Epoch (train):   0%|          | 0/432 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniconda/base/envs/catalyst/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


train (1/1) accuracy01: 0.0 | accuracy01/std: 0.0 | accuracy03: 0.0 | accuracy03/std: 0.0 | accuracy05: 0.0 | accuracy05/std: 0.0 | f1/_macro: 0.0 | f1/_micro: 0.0 | f1/_weighted: 0.0 | loss: 1977.2614941505196 | loss/mean: 1977.2614941505196 | loss/std: 1075.9956441243228 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.0 | precision/_micro: 0.0 | precision/_weighted: 0.0 | recall/_macro: 0.0 | recall/_micro: 0.0 | recall/_weighted: 0.0


/opt/homebrew/Caskroom/miniconda/base/envs/catalyst/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AssertionError: 

In [ ]:
best_val_loss = torch.inf
for epoch in range(10):
    print(f"Train epoch {epoch}")
    model.train()
    for features, targets in dm.train_dataloader():
        optimizer.zero_grad()

        predictions = model(features)
        loss = torch.sqrt(torch.mean((targets - predictions)**2)) # (4)!
        loss.backward()
        print(f"Epoch: {epoch}\tTraining loss: {loss}")

        optimizer.step()

    print(f"Validate epoch {epoch}")
    model.eval()
    val_loss = 0
    num_samples = 0
    for features, targets in dm.val_dataloader():
        predictions = model(features)
        loss = torch.sum((targets - predictions)**2)
        val_loss += loss.detach()
        num_samples += predictions.shape[0]
    val_loss = torch.sqrt(val_loss / num_samples) # (5)!

    if best_val_loss < val_loss:
        break
    else:
        best_val_loss = val_loss
        print(f"Validation loss: {best_val_loss}")

test_loss = 0
num_samples = 0
for features, targets in dm.test_dataloader():
    predictions = model(features)
    loss = torch.sqrt(torch.dist(predictions, targets))
    test_loss += loss.detach()
    num_samples += predictions.shape[0]
test_loss = torch.sqrt(test_loss / num_samples) # (6)!

print(f"Test loss: {test_loss}")